# Do stuff with SENEC CSVs with solar panel data 

In [ ]:
import pandas as pd
import datetime as dt
import glob
import os
import numpy as np
#from pandas import ExcelWriter
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## Import

To be able to loop through directory with csv's

In [ ]:
path = r'/Users/huijn001/Desktop/test/senec/'          # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

Turn csv into pandas df and concatenate all df's to one (they all have the exact same columns, so pd.concat suffices):

In [ ]:
df_from_each_file = (pd.read_csv(f, decimal=",", sep=';') for f in all_files)
df = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
print(df.head())

## Make a new dataframe from the relevant columns

Add all relevant columns in new dataframe df1 and add data as numerical data (my SENEC data is in German)

In [ ]:
df1 = df[["Stromverbrauch [kW]", "Stromerzeugung [kW]", "Netzbezug [kW]", "Netzeinspeisung [kW]"]].apply(pd.to_numeric)

Add date column to df1 as datetime

In [ ]:
df1["datum"] = pd.to_datetime(df["Uhrzeit"], format ="%d.%m.%Y %H:%M:%S")

Turn datetime column into index

In [ ]:
df1 = df1.set_index("datum")

In [ ]:
print(df1.head())

### Electric power per day

SENEC data comes in 5-minute intervals. Here we calculate data per day by adding daily data divided by 12. Stored in new df called power_per_day

In [ ]:
power_per_day = df1.resample("d").sum()

In [ ]:
print(power_per_day.shape)

In [ ]:
print(power_per_day.head())

Dividing power_per_day data by 12

In [ ]:
power_per_day.iloc[:, :] = power_per_day.iloc[:, :].div(12)

Change column names from kW to kWh

In [ ]:
power_per_day.columns = ['Stromverbrauch [kWh]', 'Stromerzeugung [kWh]', 'Netzbezug [kWh]', 'Netzeinspeisung [kWh]']

In [ ]:
#print(power_per_day.head())

Add column which substracts used and sold power from generated and bought power. If the result is more than 0 this means that more power is used than appears from the used and sold columns. This must have gone into our EV.

In [ ]:
power_per_day['Stromverbrauch PkW [kWh]'] = power_per_day['Stromerzeugung [kWh]'] + power_per_day['Netzbezug [kWh]'] - power_per_day['Stromverbrauch [kWh]'] - power_per_day['Netzeinspeisung [kWh]']

Set values 'Stromverbrauch PkW [kWh]' for 14-01-2021 and 15-01-2021 to 0 (because these values were the result of the first charge of and discharge of the battery and, therefore, not representative)

In [ ]:
power_per_day.loc[['2021-01-14', '2021-01-15'], 'Stromverbrauch PkW [kWh]'] = 0

Round new column to 0

In [ ]:
power_per_day['Stromverbrauch PkW [kWh]'] = power_per_day['Stromverbrauch PkW [kWh]'].round(0)

In [ ]:
#print(power_per_day.head())

Some statistics

In [ ]:
#power_per_day.describe()

### Power per week, month, year

Electric power per week in power_per_week

In [ ]:
power_per_week = power_per_day.resample('W-MON').sum()

In [ ]:
#print(power_per_week.head())

Electric power per month in power_per_month

In [ ]:
power_per_month = power_per_day.resample("m").sum()

In [ ]:
#power_per_month.describe()

In [ ]:
#print(power_per_month.head())

Electric power per year in power_per_year

In [ ]:
power_per_year = power_per_day.resample("y").sum()

In [ ]:
#print(power_per_year.head())

## Dataframe -> excel

### Option 1

These lines write the results of power_per_day etc. to a new excel file

In [ ]:
#writer = ExcelWriter('/path/to/new_file.xlsx')
#power_per_day.to_excel(writer,'Sheet5')
#writer.save() 

### Option 2

These lines write the results of the chosen df's to an already existing excel file (to new sheets named as such). If a sheet with the given name already exists, it won't overwrite but add a new column named existing_sheet1. 

In [ ]:
book = load_workbook('/Users/huijn001/Desktop/pv-anlage.xlsx')
writer = pd.ExcelWriter('/Users/huijn001/Desktop/pv-anlage.xlsx', engine='openpyxl') 
writer.book = book

## ExcelWriter for some reason uses writer.sheets to access the sheet.
## If you leave it empty it will not know that sheet Main is already there
## and will create a new sheet.

writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

power_per_day.to_excel(writer, "stroom per dag")
power_per_month.to_excel(writer, "stroom per maand")
power_per_year.to_excel(writer, "stroom per jaar")

writer.save()

## Visualisations

Define relevant columns for stacked bar chart

In [ ]:
bar1 = power_per_month[['Stromverbrauch [kWh]', 'Netzeinspeisung [kWh]', 'Stromverbrauch PkW [kWh]']]
bar2 = power_per_month[['Stromerzeugung [kWh]', 'Netzbezug [kWh]']]

In [ ]:
bar3 = power_per_year[['Stromverbrauch [kWh]', 'Netzeinspeisung [kWh]', 'Stromverbrauch PkW [kWh]']]
bar4 = power_per_year[['Stromerzeugung [kWh]', 'Netzbezug [kWh]']]

Define relevant columns for line chart

In [ ]:
line6 = power_per_day['Stromverbrauch [kWh]']
line7 = power_per_day['Netzbezug [kWh]']
line8 = power_per_day['Stromerzeugung [kWh]']
line9 = power_per_day['Netzeinspeisung [kWh]']
line10 = power_per_day['Stromverbrauch PkW [kWh]']

In [ ]:
line1 = power_per_week['Stromverbrauch [kWh]']
line2 = power_per_week['Netzbezug [kWh]']
line3 = power_per_week['Stromerzeugung [kWh]']
line4 = power_per_week['Netzeinspeisung [kWh]']
line5 = power_per_week['Stromverbrauch PkW [kWh]']

Index to string with relevant info for month (month-year) and year (year). This should go easier with ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y')) while creating the graph, but this gave me an error

In [ ]:
bar1.index = power_per_month.index.strftime('%b %Y')
bar2.index = power_per_month.index.strftime('%b %Y')

In [ ]:
bar3.index = power_per_year.index.strftime('%Y')
bar4.index = power_per_year.index.strftime('%Y')

Define figure, axes and style (sns.set)

In [ ]:
sns.set()

Plot a line chart for power_per_day:

In [ ]:
fig = plt.figure(figsize = (15,8))
ax = fig.add_subplot(111)

plt.plot(power_per_day.index, line6, color='orange', label='Stromverbauch', marker='.')
plt.plot(power_per_day.index, line8, color='yellow', marker='.', label='Stromerzeugung')
plt.plot(power_per_day.index, line7, color='darkgrey', marker='.', label='Netzbezug')
plt.plot(power_per_day.index, line9, color='lightgrey', marker='.', label='Netzeinspeisung')
plt.plot(power_per_day.index, line10, color='blue', marker='.', label='Stromverbrauch PkW')
plt.axhline(y=21.8, color='r', linestyle=':', label='Stromverbrauch pro Tag in 2020 (21,8 kWh)')


plt.title("Stromverbrauch pro Tag", fontsize=20)
plt.xlabel("Tag", fontsize=14)
plt.xticks(rotation=45)

# x-axis from January 2021 to January 2022
#datemin = np.datetime64(stroom_per_dag.index[0], 'Y')
#datemax = np.datetime64(stroom_per_dag.index[-1], 'Y') + np.timedelta64(1, 'Y')
#limit = ax.set_xlim(datemin, datemax)

# interval-argument makes sure that x-axis labels are only shown at given interval
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=3))

# Minor ticks every day
fmt_day = mdates.DayLocator()
ax.xaxis.set_minor_locator(fmt_day)

plt.ylabel("kWh", fontsize=14)
plt.legend()

Plot a line chart for power_per_week

In [ ]:
fig = plt.figure(figsize = (15,6))
ax = fig.add_subplot(111)

plt.plot(power_per_week.index, line1, color='orange', marker='.', label='Stromverbauch')
plt.plot(power_per_week.index, line3, color='yellow', marker='.', label='Stromerzeugung')
plt.plot(power_per_week.index, line2, color='darkgrey', marker='.', label='Netzbezug')
plt.plot(power_per_week.index, line4, color='lightgrey', marker='.', label='Netzeinspeisung')
plt.plot(power_per_week.index, line5, color='blue', marker='.', label='Stromverbrauch PkW')
plt.axhline(y=134, color='r', linestyle=':', label='Stromverbrauch pro Woche in 2020')

plt.title("Stromverbrauch pro Woche", fontsize=20)
plt.xlabel("Woche", fontsize=14)
plt.xticks(rotation=45)
plt.ylabel("kWh", fontsize=14)
plt.legend()

Plot a stacked bar chart for monthly and yearly data

In [ ]:
fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)

bar1.plot(kind="bar", stacked=True, width=0.3, 
                  ax=ax, position=0, rot=45, color=sns.color_palette("muted"))
bar2.plot(kind="bar", stacked=True, width=0.3, 
                   ax=ax, position=1, hatch='..', rot=0, color=sns.color_palette("pastel"))

plt.xlabel('Date', fontsize=14)
plt.ylabel('kWh', fontsize=14)
plt.title('Stromverbrauch pro Monat', fontsize=20)
#plt.legend('upper left')

#ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))

#plt.savefig('/Users/huijn001/Desktop/test/stroom_per_maand.png', dpi=300, bbox_inches='tight') # save chart as png
plt.show()

In [ ]:
fig1 = plt.figure(figsize = (10,6))
ax = fig1.add_subplot(111)
# pal = ["#9b59b6", "#2ecc71"]
# "#34495e", "#e74c3c"

bar3.plot(kind="bar", stacked=True, width=0.1, 
                  ax=ax, position=0, rot=45, color=sns.color_palette("muted"))
bar4.plot(kind="bar", stacked=True, width=0.1, 
                   ax=ax, position=1, hatch='..', alpha=0.7, rot=0, color=sns.color_palette("pastel"))

plt.xlabel('Date', fontsize=14)
plt.ylabel('kWh', fontsize=14)
plt.title('Stromverbrauch pro Jahr', fontsize=20)

#ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

#plt.savefig('/Users/huijn001/Desktop/test/stroom_per_jaar.png', dpi=300, bbox_inches='tight') # save chart as png
plt.show()

Generate a heatmap of the correlation between the different columns with df.corr() and sns.heatmap. Green stands for a positive and red for a negative correlation

In [ ]:
sns.heatmap(power_per_day.corr(), square=True,cmap='RdYlGn')